# Aula 7: Estruturação do banco e JOIN us ON su.c = e.sso;

In [1]:
# !pip install pandas
# !pip install pandasql
# !pip install sqlalchemy==1.4.46

In [2]:
import pandas as pd
import pandasql as ps
import os

In [3]:
# Carregando os dados

vendedores = pd.read_csv("base_curso/vendedores.csv")
transacoes = pd.read_csv("base_curso/transacoes.csv")
pagamentos = pd.read_csv("base_curso/pagamento.csv")
cursos = pd.read_csv("base_curso/cursos.csv")
cidades = pd.read_csv("base_curso/cidades.csv")
alunos = pd.read_csv("base_curso/alunos.csv")

In [4]:
ps.sqldf("""
SELECT
  *
FROM
  transacoes
""")

,id_transacao,id_aluno,id_curso,id_vendedor,id_pagamento
0,1,1,1,1,1
1,2,2,1,2,2
2,3,3,1,1,2
3,4,4,1,2,1
4,5,5,1,1,3
5,6,6,1,1,3
6,7,7,1,2,1
7,8,8,1,2,1
8,9,9,2,1,4
9,10,10,2,2,4


In [5]:
# Integrar as tabelas vendedores e transações

ps.sqldf("""
SELECT
  *
FROM
  vendedores AS V
  LEFT JOIN transacoes AS T ON V.id_vendedor = T.id_vendedor
""")

,id_vendedor,vendedor,id_transacao,id_aluno,id_curso,id_vendedor,id_pagamento
0,1,paulo,1,1,1,1,1
1,1,paulo,3,3,1,1,2
2,1,paulo,5,5,1,1,3
3,1,paulo,6,6,1,1,3
4,1,paulo,9,9,2,1,4
5,1,paulo,14,14,2,1,1
6,1,paulo,15,15,2,1,5
7,1,paulo,16,16,2,1,2
8,1,paulo,17,17,2,1,2
9,1,paulo,21,21,2,1,1


In [6]:
# 1. Quantos cursos cada vendedor vendeu?

ps.sqldf("""
SELECT
  V.vendedor,
  COUNT(C.valor_curso) AS qtdade_vendida
FROM
  transacoes AS T
  LEFT JOIN vendedores AS V ON T.id_vendedor = V.id_vendedor
  LEFT JOIN cursos AS C ON T.id_curso = C.id_curso
GROUP BY
  V.vendedor
""")

,vendedor,qtdade_vendida
0,adriana,13
1,nadir,7
2,paulo,14


In [7]:
# 2. Qual valor cada vendedor vendeu?

ps.sqldf("""
SELECT
  V.vendedor,
  SUM(C.valor_curso) AS montante
FROM
  transacoes AS T
  LEFT JOIN vendedores AS V ON T.id_vendedor = V.id_vendedor
  LEFT JOIN cursos AS C ON T.id_curso = C.id_curso
GROUP BY
  V.vendedor
""")

,vendedor,montante
0,adriana,920
1,nadir,410
2,paulo,860


In [8]:
# 3. Quantos de cada curso foi vendido por cada vendedor?

ps.sqldf("""
SELECT
  V.vendedor,
  C.curso,
  COUNT(C.valor_curso) AS quantidade
FROM
  transacoes AS T
  LEFT JOIN vendedores AS V ON T.id_vendedor = V.id_vendedor
  LEFT JOIN cursos AS C ON T.id_curso = C.id_curso
GROUP BY
  V.vendedor,
  C.curso
""")

,vendedor,curso,quantidade
0,adriana,arquiteto dados,5
1,adriana,ciencia dados,4
2,adriana,engenharia dados,4
3,nadir,analise dados,4
4,nadir,ciencia dados,3
5,paulo,analise dados,4
6,paulo,ciencia dados,6
7,paulo,engenharia dados,4


In [9]:
# 4. Qual o montante de cada curso arrecadado por cada vendedor?

ps.sqldf("""
SELECT
  V.vendedor,
  C.curso,
  SUM(C.valor_curso) AS montante
FROM
  transacoes AS T
  LEFT JOIN vendedores AS V ON T.id_vendedor = V.id_vendedor
  LEFT JOIN cursos AS C ON T.id_curso = C.id_curso
GROUP BY
  V.vendedor,
  C.curso
""")

,vendedor,curso,montante
0,adriana,arquiteto dados,400
1,adriana,ciencia dados,280
2,adriana,engenharia dados,240
3,nadir,analise dados,200
4,nadir,ciencia dados,210
5,paulo,analise dados,200
6,paulo,ciencia dados,420
7,paulo,engenharia dados,240


# Dica "inFAOlível" da ordem das operações na query

- F: Filtra -> WHERE
- A: Agrega -> GROUP BY
- O: Ordena -> ORDER BY

### Exercício 1:

Realize uma query com a média do recebido por curso

In [10]:
ps.sqldf("""
SELECT
  AVG(C.valor_curso) AS Média
FROM
  transacoes AS T
  LEFT JOIN cursos AS C ON T.id_curso = C.id_curso
""")

,Média
0,64.411765


### Exercício 2:

Realize uma query que calcule o quanto foi recebido de alunos que começam com a letra A;

In [11]:
ps.sqldf("""
SELECT
  SUM(C.valor_curso) AS Total
FROM
  transacoes AS T
  LEFT JOIN alunos AS A ON T.id_aluno = A.id_aluno
  LEFT JOIN cursos AS C ON T.id_curso = C.id_curso
WHERE
  primeiro_nome LIKE 'a%'
""")

,Total
0,300


### Exercício 3:

Realize uma query que calcule quanto foi recebido de cada cidade


E se eu quiser ver resultados acima de um valor específico? Como filtro o Group By?

In [12]:
ps.sqldf("""
SELECT
  cidade,
  SUM(C.valor_curso) AS Total
FROM
  transacoes AS T
  LEFT JOIN alunos AS A ON T.id_aluno = A.id_aluno
  LEFT JOIN cidades AS C ON A.id_cidade = C.id_cidade
  LEFT JOIN cursos AS C ON T.id_curso = C.id_curso
GROUP BY
  cidade
""")

,cidade,Total
0,alto paraíso,160
1,congonhas,150
2,curitiba,520
3,jaru,130
4,jequié,210
5,ouro preto,160
6,porto velho,150
7,salvador,280
8,sao paulo,240
9,tiradentes,70


In [13]:
ps.sqldf("""
SELECT
  cidade,
  SUM(C.valor_curso) AS Total
FROM
  transacoes AS T
  LEFT JOIN alunos AS A ON T.id_aluno = A.id_aluno
  LEFT JOIN cidades AS C ON A.id_cidade = C.id_cidade
  LEFT JOIN cursos AS C ON T.id_curso = C.id_curso
WHERE
  cidade LIKE 'curitiba'
""")

,cidade,Total
0,curitiba,520


### Exercício 4:

Realize uma query que calcule o quanto foi recebido por cada estado.

In [14]:
ps.sqldf("""
SELECT
  estado,
  SUM(C.valor_curso) AS Total
FROM
  transacoes AS T
  LEFT JOIN alunos AS A ON T.id_aluno = A.id_aluno
  LEFT JOIN cidades AS C ON A.id_cidade = C.id_cidade
  LEFT JOIN cursos AS C ON T.id_curso = C.id_curso
GROUP BY
  estado
""")

,estado,Total
0,BA,490
1,MG,430
2,PR,520
3,RO,440
4,SP,240
5,mg,70


### Exercício 5:

Realize queryes para saber quem trouxe mais dinheiro: pessoas com dois telefones ou com um telefone?

In [15]:
print("Somatório de pessoas com dois telefones: ")
ps.sqldf("""
SELECT
  SUM(C.valor_curso) AS Total
FROM
  transacoes AS T
  LEFT JOIN alunos AS A ON T.id_aluno = A.id_aluno
  LEFT JOIN cursos AS C ON T.id_curso = C.id_curso
WHERE
  telefone_b != 0
""")

Somatório de pessoas com dois telefones: 


,Total
0,1560


In [16]:
print("Somatório de pessoas com um telefones: ")
ps.sqldf("""
SELECT
  SUM(C.valor_curso) AS Total
FROM
  transacoes AS T
  LEFT JOIN alunos AS A ON T.id_aluno = A.id_aluno
  LEFT JOIN cursos AS C ON T.id_curso = C.id_curso
WHERE
  telefone_b == 0
""")

Somatório de pessoas com um telefones: 


,Total
0,630


### Exercício 6:

Realize uma query para saber qual foi o lucro total por cada vendedor

In [17]:
ps.sqldf("""
SELECT
  vendedor,
  SUM(C.valor_curso) AS Total
FROM
  transacoes AS T
  LEFT JOIN vendedores AS V ON T.id_vendedor = V.id_vendedor
  LEFT JOIN cursos AS C ON T.id_curso = C.id_curso
GROUP BY
  vendedor
""")

,vendedor,Total
0,adriana,920
1,nadir,410
2,paulo,860


### Exercício 7:

Realize uma query para saber o valor médio das vendas de cada vendedor

In [18]:
ps.sqldf("""
SELECT
  vendedor,
  AVG(C.valor_curso) AS Média
FROM
  transacoes AS T
  LEFT JOIN vendedores AS V ON T.id_vendedor = V.id_vendedor
  LEFT JOIN cursos AS C ON T.id_curso = C.id_curso
GROUP BY
  vendedor
""")

,vendedor,Média
0,adriana,70.769231
1,nadir,58.571429
2,paulo,61.428571


### Exercicio 8:

Qual vendedor vendeu mais cursos?

In [19]:
ps.sqldf("""
SELECT
  vendedor,
  COUNT (curso) AS qtdade_vendas
FROM
  transacoes AS T
  LEFT JOIN vendedores AS V ON T.id_vendedor = V.id_vendedor
  LEFT JOIN cursos AS C ON T.id_curso = C.id_curso
GROUP BY
  vendedor
ORDER BY
  qtdade_vendas DESC
""")

,vendedor,qtdade_vendas
0,paulo,14
1,adriana,13
2,nadir,7


### Exercício 9:

Quais os tipos de pagamentos mais usado por cada vendedor?

In [20]:
ps.sqldf("""
SELECT
  vendedor,
  pagamento,
  COUNT (pagamento) AS qtdade_pgto
FROM
  transacoes AS T
  LEFT JOIN vendedores AS V ON T.id_vendedor = V.id_vendedor
  LEFT JOIN pagamentos AS P ON T.id_pagamento = P.id_pagamento
GROUP BY
  vendedor,
  pagamento
ORDER BY
  vendedor,
  qtdade_pgto DESC
""")

,vendedor,pagamento,qtdade_pgto
0,adriana,credito,5
1,adriana,a vista,5
2,adriana,crediário,2
3,adriana,carnê,1
4,nadir,crediário,3
5,nadir,pix,2
6,nadir,carnê,2
7,paulo,credito,5
8,paulo,crediário,3
9,paulo,a vista,3


### Exercício 10:

Quais tipos de pagamentos são mais utilizados?

In [21]:
ps.sqldf("""
SELECT
  pagamento,
  COUNT (pagamento) AS qtdade_pgto
FROM
  transacoes AS T
  LEFT JOIN pagamentos AS P ON T.id_pagamento = P.id_pagamento
GROUP BY
  pagamento
ORDER BY
  qtdade_pgto DESC
""")

,pagamento,qtdade_pgto
0,credito,10
1,crediário,8
2,a vista,8
3,pix,4
4,carnê,4


### Exercício 11:

Quais pagamentos levantam mais montante de dinheiro?

In [22]:
ps.sqldf("""
SELECT
  pagamento,
  SUM (valor_curso) AS Total
FROM
  transacoes AS T
  LEFT JOIN cursos AS C ON T.id_curso = C.id_curso
  LEFT JOIN pagamentos AS P ON T.id_pagamento = P.id_pagamento
GROUP BY
  pagamento
ORDER BY
  Total DESC
""")

,pagamento,Total
0,credito,670
1,a vista,520
2,crediário,500
3,pix,260
4,carnê,240


### Exercício 12:

Quais pagamentos apresentam os maiores valores médio?

In [23]:
ps.sqldf("""
SELECT
  pagamento,
  AVG (valor_curso) AS Média
FROM
  transacoes AS T
  LEFT JOIN cursos AS C ON T.id_curso = C.id_curso
  LEFT JOIN pagamentos AS P ON T.id_pagamento = P.id_pagamento
GROUP BY
  pagamento
ORDER BY
  Média DESC
""")

,pagamento,Média
0,credito,67.0
1,pix,65.0
2,a vista,65.0
3,crediário,62.5
4,carnê,60.0


### Exercício 13:

Quais as cidades que cada vendedor atua mais?

In [24]:
ps.sqldf("""
SELECT
  vendedor,
  cidade,
  COUNT(vendedor) AS Total
FROM
  transacoes AS T
  LEFT JOIN vendedores AS v ON T.id_vendedor = V.id_vendedor
  LEFT JOIN alunos AS A ON T.id_aluno = A.id_aluno
  LEFT JOIN cidades AS C ON A.id_cidade = C.id_cidade
GROUP BY
  vendedor,
  cidade
ORDER BY
  vendedor ASC,
  Total DESC
""")

,vendedor,cidade,Total
0,adriana,salvador,3
1,adriana,sao paulo,2
2,adriana,ouro preto,2
3,adriana,curitiba,2
4,adriana,alto paraíso,2
5,adriana,jequié,1
6,adriana,jaru,1
7,nadir,congonhas,3
8,nadir,curitiba,2
9,nadir,uberlandia,1


### Exercício 14:

Qual o curso preferido de cada cidade?

In [25]:
ps.sqldf("""
SELECT
  cidade,
  curso,
  Total
FROM
  (SELECT
    cidade,
    curso,
    COUNT (curso) AS Total,
    ROW_NUMBER() OVER (PARTITION BY cidade ORDER BY COUNT(curso) DESC) AS first
  FROM
    transacoes AS T
    LEFT JOIN alunos AS A ON T.id_aluno = A.id_aluno
    LEFT JOIN cidades AS C ON A.id_cidade = C.id_cidade
    LEFT JOIN cursos AS C ON T.id_curso = C.id_curso
  GROUP BY
    cidade,
    curso
  ORDER BY
    cidade ASC,
    total DESC)
WHERE
  first = 1
ORDER BY
  cidade ASC
""")

,cidade,curso,Total
0,alto paraíso,arquiteto dados,2
1,congonhas,analise dados,3
2,curitiba,engenharia dados,4
3,jaru,arquiteto dados,1
4,jequié,ciencia dados,3
5,ouro preto,arquiteto dados,2
6,porto velho,analise dados,3
7,salvador,ciencia dados,4
8,sao paulo,engenharia dados,4
9,tiradentes,ciencia dados,1


### Exercício 15:

Qual o montante mensal recebido?

In [35]:
# NO COLAB ELE ACEITA A FUNÇÃO SUBSTRING

# ps.sqldf("""
# SELECT
#   SUBSTRING(data_ingresso, 6, 2) AS month,
#   SUM(valor_curso) AS sum_valor_curso
# FROM
#   transacoes AS T
#   LEFT JOIN alunos AS A ON T.id_aluno = A.id_aluno
#   LEFT JOIN cursos AS C ON T.id_curso = C.id_curso
# GROUP BY
#   month
# """)
         
# NO JUPYTER PRECISEI ALTERAR PARA SUBSTR

ps.sqldf("""
SELECT
  SUBSTR(data_ingresso, 6, 2) AS month,
  SUM(valor_curso) AS sum_valor_curso
FROM
  transacoes AS T
  LEFT JOIN alunos AS A ON T.id_aluno = A.id_aluno
  LEFT JOIN cursos AS C ON T.id_curso = C.id_curso
GROUP BY
  month
""")

,month,sum_valor_curso
0,03,1690
1,04,500


### Exercício 16:

Qual a média mensal recebida?

In [ ]:
ps.sqldf("""
SELECT
  SUBSTRING(data_ingresso, 6, 2) AS month,
  AVG(valor_curso) AS sum_valor_curso
FROM
  transacoes AS T
  LEFT JOIN alunos AS A ON T.id_aluno = A.id_aluno
  LEFT JOIN cursos AS C ON T.id_curso = C.id_curso
GROUP BY
  month
""")

,month,sum_valor_curso
0,03,62.592593
1,04,71.428571


### Exerício 17:

Dos alunos com dois telefones, qual montante levantado por cada tipo de pagamento?

In [ ]:
ps.sqldf("""
SELECT
  pagamento,
  SUM (valor_curso) AS total
FROM
  transacoes AS T
  LEFT JOIN alunos AS A ON T.id_aluno = A.id_aluno
  LEFT JOIN cursos AS C ON T.id_curso = C.id_curso
  LEFT JOIN pagamentos AS P ON T.id_pagamento = P.id_pagamento
WHERE
  telefone_b != 0
GROUP BY
  pagamento
ORDER BY
  total DESC
""")

,pagamento,total
0,a vista,390
1,crediário,380
2,credito,360
3,pix,260
4,carnê,170


### Exercício 18:

Quais os cursos mais frequentes ?

In [ ]:
ps.sqldf("""
SELECT
  curso,
  COUNT (curso) AS total
FROM
  transacoes AS T
  LEFT JOIN cursos AS C ON T.id_curso = C.id_curso
GROUP BY
  curso
ORDER BY
  total DESC
""")

,curso,total
0,ciencia dados,13
1,engenharia dados,8
2,analise dados,8
3,arquiteto dados,5


### Exercício 19:

Quantos alunos temos por cada cidade? Mais alunos na cidade significa maior montante gerado pela cidade?

In [ ]:
ps.sqldf("""
SELECT
  cidade,
  COUNT(cidade) AS alunos_cidade,
  SUM(valor_curso) AS montante
FROM
  transacoes AS T
  LEFT JOIN alunos AS A ON T.id_aluno = A.id_aluno
  LEFT JOIN cidades AS C ON A.id_cidade = C.id_cidade
  LEFT JOIN cursos AS C ON T.id_curso = C.id_curso
GROUP BY
  cidade
ORDER BY
  montante DESC
""")

,cidade,alunos_cidade,montante
0,curitiba,8,520
1,salvador,4,280
2,sao paulo,4,240
3,jequié,3,210
4,ouro preto,2,160
5,alto paraíso,2,160
6,porto velho,3,150
7,congonhas,3,150
8,jaru,2,130
9,uberlandia,2,120


In [ ]:
print("Não necessariamente quanto mais alunos na cidade significa maior montante gerado pela cidade! Por exemplo:\n Ouro Preto -> 2 alunos = R$ 160\n Porto Velho -> 3 alunos = R$ 150")

Não necessariamente quanto mais alunos na cidade significa maior montante gerado pela cidade! Por exemplo:
 Ouro Preto -> 2 alunos = R$ 160
 Porto Velho -> 3 alunos = R$ 150


### Exercício 20:

Quais os montates levantados pelos alunos que contém a letra G no nome?

In [ ]:
ps.sqldf("""
SELECT
  SUM(C.valor_curso) AS Total
FROM
  transacoes AS T
  LEFT JOIN alunos AS A ON T.id_aluno = A.id_aluno
  LEFT JOIN cursos AS C ON T.id_curso = C.id_curso
WHERE
  primeiro_nome LIKE '%g%' OR
  ultimo_nome LIKE '%g%'
""")

,Total
0,440
